# Exploratory Data Analysis #1

### Import libraries

In [2]:
import pandas as pd
from surprise import Reader, Dataset

### Import data

In [3]:
df2 = pd.read_csv('comb_df7.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54387 entries, 0 to 54386
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        54387 non-null  int64  
 1   genre             54387 non-null  object 
 2   title             54387 non-null  object 
 3   author            54387 non-null  object 
 4   pages             54387 non-null  int64  
 5   average_rating    54387 non-null  float64
 6   book_description  40773 non-null  object 
 7   user_unique_key   54387 non-null  int64  
 8   number_of_stars   54387 non-null  int64  
 9   book_unique_key   54387 non-null  int64  
 10  review_text       53340 non-null  object 
dtypes: float64(1), int64(5), object(5)
memory usage: 4.6+ MB


In [5]:
reader = Reader()
data = Dataset.load_from_df(df2,reader)

ValueError: too many values to unpack (expected 3)

In [31]:
new_df = df2.drop('genre','title','author', 'pages', 'average_rating', 'book_description', 'review_text')

In [32]:
reader = Reader()
data = Dataset.load_from_df(new_df,reader)

AttributeError: 'DataFrame' object has no attribute 'itertuples'

### Inspect the data

### Pre-process the data

In [ ]:
#join the tables

In [ ]:
#check for nulls/nans

### EDA visualizations

### Build first simple model (FSM)

### FSM visualizations

In [38]:

from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.recommendation import ALS

from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import pyspark
spark = SparkSession.builder.getOrCreate()

from pyspark.sql.types import IntegerType
from pyspark.sql.functions import isnan, when, count, col


In [7]:
import os
import pandas as pd

In [42]:
dfpd= pd.read_csv('comb_df7.csv')

In [47]:
dfpd['user_unique_key'].nunique()

27642

In [10]:
df2 = spark.read.csv('comb_df7.csv', header='true', inferSchema='true')

In [11]:
df2.dtypes

[('_c0', 'string'),
 ('genre', 'string'),
 ('title', 'string'),
 ('author', 'string'),
 ('pages', 'string'),
 ('average_rating', 'string'),
 ('book_description', 'string'),
 ('user_unique_key', 'string'),
 ('number_of_stars', 'string'),
 ('book_unique_key', 'string'),
 ('review_text', 'string')]

In [13]:
output_df = df2.withColumn("number_of_stars",df2["number_of_stars"].cast(IntegerType()))

In [14]:
output_df.dtypes

[('_c0', 'string'),
 ('genre', 'string'),
 ('title', 'string'),
 ('author', 'string'),
 ('pages', 'string'),
 ('average_rating', 'string'),
 ('book_description', 'string'),
 ('user_unique_key', 'string'),
 ('number_of_stars', 'int'),
 ('book_unique_key', 'string'),
 ('review_text', 'string')]

In [16]:
output2 = output_df.withColumn('book_unique_key', output_df['book_unique_key'].cast(IntegerType()))

In [17]:
output2.dtypes

[('_c0', 'string'),
 ('genre', 'string'),
 ('title', 'string'),
 ('author', 'string'),
 ('pages', 'string'),
 ('average_rating', 'string'),
 ('book_description', 'string'),
 ('user_unique_key', 'string'),
 ('number_of_stars', 'int'),
 ('book_unique_key', 'int'),
 ('review_text', 'string')]

In [19]:
out3= output2.withColumn('user_unique_key', output2['user_unique_key'].cast(IntegerType()))

In [48]:
out3.dtypes

[('_c0', 'string'),
 ('genre', 'string'),
 ('title', 'string'),
 ('author', 'string'),
 ('pages', 'string'),
 ('average_rating', 'string'),
 ('book_description', 'string'),
 ('user_unique_key', 'int'),
 ('number_of_stars', 'int'),
 ('book_unique_key', 'int'),
 ('review_text', 'string')]

In [50]:
out3.where(out3['user_unique_key'].isNull()).select('user_unique_key','title').show()

+---------------+--------------------+
|user_unique_key|               title|
+---------------+--------------------+
|           null|The Neil Gaiman R...|
|           null| chosen by those ...|
|           null| endlessly imagin...|
|           null|               11931|
|           null|The Neil Gaiman R...|
|           null| chosen by those ...|
|           null| endlessly imagin...|
|           null|               26212|
|           null|                null|
|           null|           so clever|
|           null|                null|
|           null|The Neil Gaiman R...|
|           null| chosen by those ...|
|           null| endlessly imagin...|
|           null|               23480|
|           null|The Neil Gaiman R...|
|           null| chosen by those ...|
|           null| endlessly imagin...|
|           null|               19564|
|           null|The Neil Gaiman R...|
+---------------+--------------------+
only showing top 20 rows



In [36]:
out3.summary().show()

+-------+--------------------+--------------------+--------------------+-----------------+------------------+--------------------+------------------+----------------+------------------+------------------+------------------+
|summary|                 _c0|               genre|               title|           author|             pages|      average_rating|  book_description| user_unique_key|   number_of_stars|   book_unique_key|       review_text|
+-------+--------------------+--------------------+--------------------+-----------------+------------------+--------------------+------------------+----------------+------------------+------------------+------------------+
|  count|              265816|              191600|              162269|           141960|            126761|              109657|             82504|           25628|             23207|             21477|             27168|
|   mean|   3563.036253465783|   12870.39360988052|   5592.702028706599|5332.570139968896|1250.619848042

In [20]:
train, test = out3.randomSplit([0.8, 0.2])

In [37]:
als = ALS(maxIter=5,rank=4, 
          userCol='user_unique_key', itemCol='book_unique_key', 
          ratingCol='number_of_stars',
          coldStartStrategy='drop')


model_fsm = als.fit(train)

Py4JJavaError: An error occurred while calling o193.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 28, 10.0.0.8, executor driver): org.apache.spark.SparkException: Failed to execute user defined function(ALSModelParams$$Lambda$2509/877581283: (int) => int)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:729)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$SliceIterator.hasNext(Iterator.scala:266)
	at scala.collection.Iterator.for